In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")


In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_dataportal',
    'd_portal_gd_concept_region_sku_tet',
    proc_date,
    'temp_d_portal_gd_concept_region_sku_tet'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_dataportal.d_portal_gd_concept_region_sku_tet
(
  giai_doan STRING,
  concept STRING,
  region_province STRING,
  product_id STRING,
  product_name STRING,
  is_delete STRING,
  proc_date DATE,
  upload_user STRING,
  upload_time TIMESTAMP,
  file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
source_columns = spark.table("temp_d_portal_gd_concept_region_sku_tet").columns
source_columns.remove("file_creation_ts")
insert_row = ", ".join(source_columns)

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_silver_dataportal.d_portal_gd_concept_region_sku_tet
WHERE proc_date = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_dataportal.d_portal_gd_concept_region_sku_tet ({insert_row})
SELECT {insert_row} FROM temp_d_portal_gd_concept_region_sku_tet
""")